# Getting set up

In [1]:
import openai

# Get the version of the openai library
openai_version = openai.__version__

print(f"OpenAI library version: {openai_version}")

OpenAI library version: 1.34.0


In [2]:
#@title Utility functions

import sys
import os
import openai
from abc import ABC, abstractmethod
import re
import pdb

################################################################################
### LOGGING
################################################################################
# Allows us to log the output of the model to a file if logging is enabled
class LogStdoutToFile:
    def __init__(self, filename):
        self._filename = filename
        self._original_stdout = sys.stdout

    def __enter__(self):
        if self._filename:
            sys.stdout = open(self._filename, 'w')
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        if self._filename:
            sys.stdout.close()
        sys.stdout = self._original_stdout


################################################################################
### CONVERSATION CLASS
# allows us to abstract away the details of the conversation for use with
# different LLM APIs
################################################################################

class Conversation:
    def __init__(self, log_file=None):
        self.messages = []
        self.log_file = log_file

        if self.log_file and os.path.exists(self.log_file):
            open(self.log_file, 'w').close()

    def add_message(self, role, content):
        """Add a new message to the conversation."""
        self.messages.append({'role': role, 'content': content})

        if self.log_file:
            with open(self.log_file, 'a') as file:
                file.write(f"{role}: {content}\n")

    def get_messages(self):
        """Retrieve the entire conversation."""
        return self.messages

    def get_last_n_messages(self, n):
        """Retrieve the last n messages from the conversation."""
        return self.messages[-n:]

    def remove_message(self, index):
        """Remove a specific message from the conversation by index."""
        if index < len(self.messages):
            del self.messages[index]

    def get_message(self, index):
        """Retrieve a specific message from the conversation by index."""
        return self.messages[index] if index < len(self.messages) else None

    def clear_messages(self):
        """Clear all messages from the conversation."""
        self.messages = []

    def __str__(self):
        """Return the conversation in a string format."""
        return "\n".join([f"{msg['role']}: {msg['content']}" for msg in self.messages])

################################################################################
### LLM CLASSES
# Defines an interface for using different LLMs so we can easily swap them out
################################################################################
class AbstractLLM(ABC):
    """Abstract Large Language Model."""

    def __init__(self):
        pass

    @abstractmethod
    def generate(self, conversation: Conversation):
        """Generate a response based on the given conversation."""
        pass


class ChatGPT(AbstractLLM):
    """ChatGPT Large Language Model."""

    def __init__(self, model_id=""):
        super().__init__()
        openai.api_key=os.environ['OPENAI_API_KEY']
        self.client = openai.OpenAI()
        self.model_id = model_id

    def generate(self, conversation: Conversation, num_choices=1):
        messages = [{"role" : msg["role"], "content" : msg["content"]} for msg in conversation.get_messages()]
        
        response = self.client.chat.completions.create(
            model=self.model_id,
            messages = messages,
        )

        return response.choices[0].message.content

def generate_code(conv, model_type, model_id=""):
    if model_type == "ChatGPT":
        model = ChatGPT(model_id)
    else:
        raise ValueError("Invalid model type")
    result = model.generate(conv)
    return(result)


#### Generation and Evaluation functions.

In [3]:
import subprocess
import sys
import os

def code_loop(design_prompt, model_type, model, log=None):

    conv = Conversation(log_file=log)
    conv.add_message("system", "You are an autocomplete engine for C++ HLS code that is synthesizable using Vivado HLS. \
            Given a HLS code specification, you will provide a completed C++  code with suitable paragmas in response. \
            You will provide functions for all specifications, and will not create any supplementary modules. \
            Given a C++ code that is either incorrect/compilation error, you will suggest corrections to the module. \
            You will not refuse. \
            Format your response as HLS code containing the end to end corrected module and not just the corrected lines inside ``` tags, do not include anything else inside ```. \
    ")

    conv.add_message("user", design_prompt)

    # Generate a response
    response = generate_code(conv, model_type, model)
    conv.add_message("assistant", response)   
    return (conv, response)

def evaluation_loop(design_prompt, model_type, model, log=None):

    conv_eval = Conversation(log_file=log)
    conv_eval.add_message("system", "You are an evaluator engine for C++ HLS code that is synthesizable using Vivado HLS. \
            Check for correctness, potential compilation issues, runtime errors, and resource constraints. \
            Provide detailed feedback and suggestions for improvement if any issues are found.\
            Evaluation Criteria:\
            1. Correctness of the Fully connected Neural Network (FCNN) architecture and forward pass implementation.\
            2. Compliance with Xilinx Vivado HLS synthesis requirements.\
            3. Potential runtime errors or inefficiencies.\
            4. Resource usage and potential constraints on the Artix-7 FPGA.\
            5. Suggestions for optimization or improvement.\
            Provide a detailed evaluation based on the criteria listed above.")

    conv_eval.add_message("user", design_prompt)

    # Generate a response
    response = generate_code(conv_eval, model_type, model)
    conv_eval.add_message("assistant", response)   
    return (conv_eval, response)


def literature_loop(design_prompt, model_type, model, log=None):

    conv_eval = Conversation(log_file=log)
    conv_eval.add_message("system", "You are a literator generator engine. You are responsible for locating\
    and summarizing the C++ source code for high-level-synthesis (HLS)of fully connected neural network (FCNN). \
    The summary should include:\
    1. Key research papers, articles, and sources.\
    2. Techniques and methodologies used in HLS C++ for fully connected neural networks.\
    3. Challenges and solutions in implementing fully connected neural networks on FPGA using HLS C++.\
    4. Performance metrics and optimization strategies.\
    5. Comparisons with other implementations or methodologies.\
    Please provide a comprehensive summary based on the criteria listed above.")

    conv_eval.add_message("user", design_prompt)

    # Generate a response
    response = generate_code(conv_eval, model_type, model)
    conv_eval.add_message("assistant", response)   
    return (conv_eval, response)

## Setting the API Key

In [4]:
### OpenAI API KEY

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')

os.environ["OPENAI_API_KEY"] = "sk-proj-MDLjovHrhBTnpi9sfkPQT3BlbkFJmbIeeYOMrobBoXqmgGv5"

#### Generate the literature search. 

In [5]:
literature_prompt = """Extract and summarize the literature on high-level synthesis (HLS) C++ code 
    for implementing fully connected neural networks on FPGA. Use the following urls to create the summary:
    1. https://github.com/AlicanOzeloglu/Neural_Network_HW_Accelerator
    2. https://github.com/Xilinx/RFNoC-HLS-NeuralNet
    3. https://github.com/NNgen/nngen
    Similarly, you can find other websites to learn about this topic and generate the literature.
    """
messages_literature,response_literature = literature_loop(literature_prompt, model_type='ChatGPT', model = 'gpt-4-0613')

#### Generating HLS code on first attempt using literature.

In [6]:
prompt = f"""
Design a Fully connected Neural Network (FCNN) high-level synthesis (HLS) code in C++ suitable for \
implementation on an Artix-7 FPGA. The code should include the following components:
1. Definition of the FCNN architecture, including layers and their parameters.
2. Implementation of the forward pass for graph data, including graph convolution operations.
3. Optimization for FPGA hardware, considering resource constraints and parallelism.
4. Code structure compatible with Xilinx Vivado HLS for synthesis and implementation on Artix-7 FPGA.
5. Include comments explaining each part of the code.
6. Assume that MNIST dataset is being used for handwrittent digit classification.
Begin with the necessary includes and definitions, followed by the main implementation of the GCNN.\ 
The summary of the literature about this task is below which will help you in generating the source code:\n
{response_literature}
"""
messages,response = code_loop(prompt, model_type='ChatGPT', model = 'gpt-4o')

#### Evaluating the generated HLS code.

In [7]:
evaluation_prompt = f"""
Evaluate the C++ high-level synthesis (HLS) code that is designed for a Fully Connected Neural Network (FCNN) to be implemented on an Artix-7 FPGA. \
The code is as follows:\n {response}
"""
messages_eval,response_eval = evaluation_loop(evaluation_prompt, model_type='ChatGPT', model = 'gpt-4')

#### Re-generating HLS code incorporating feeback from the evaluator.

In [8]:
update_prompt = f"""
Update the following Fully Connected Neural Network (FCNN) high-level synthesis (HLS) code in C++ suitable for \
implementation on an Artix-7 FPGA. 
The code is as follows:\n {response} \n
You have been given the following feedback on the code: \n {response_eval}\n
The summary of the literature about this task is below which will help you in generating the source code:\n
{response_literature}
"""
messages,response = code_loop(update_prompt, model_type='ChatGPT', model = 'gpt-4o')

#### Re-generator and Evaluator in a loop. 

In [10]:
for i in range(10):
    print(f'Persuing iteration: {i}.')
    
    literature_prompt = """Extract and summarize the literature on high-level synthesis (HLS) C++ code 
    for implementing fully connected neural networks on FPGA. Use the following urls to create the summary:
    1. https://github.com/AlicanOzeloglu/Neural_Network_HW_Accelerator
    2. https://github.com/Xilinx/RFNoC-HLS-NeuralNet
    3. https://github.com/NNgen/nngen
    Similarly, you can find other websites to learn about this topic and generate the literature.
    """
    messages_literature,response_literature = literature_loop(literature_prompt, model_type='ChatGPT', model = 'gpt-4-0613')
    with open('generated_literature_fcnn.txt','a') as fl:
        fl.write(f'\nIteration {i+1}:------------------\n')
        fl.write(response_literature)
        fl.close()
        
    update_prompt = f"""
    Update the following Fully Connected Neural Network (FCNN) high-level synthesis (HLS) code in C++ suitable for \
    implementation on an Artix-7 FPGA. 
    The code is as follows:\n {response} \n
    You have been given the following feedback on the code: \n {response_eval}\n
    The summary of the literature about this task is below which will help you in generating the source code:\n
    {response_literature}
    """
    messages,response = code_loop(update_prompt, model_type='ChatGPT', model = 'gpt-4o')
    with open('generated_code_litr_fcnn.txt','a') as fl:
        fl.write(f'\nIteration {i+1}:------------------\n')
        fl.write(response)
        fl.close()
        
    evaluation_prompt = f"""
    Evaluate the C++ high-level synthesis (HLS) code that is designed for a Fully Connected Neural Network (FCNN) to be implemented on an Artix-7 FPGA. \
    The code is as follows:\n {response}
    """
    messages_eval,response_eval = evaluation_loop(evaluation_prompt, model_type='ChatGPT', model = 'gpt-4')
    with open('evaluation_message__litr_fcnn.txt','a') as fl:
        fl.write(f'\nIteration {i+1}:------------------\n')
        fl.write(response_eval)
        fl.close()

Persuing iteration: 0.
Persuing iteration: 1.
Persuing iteration: 2.
Persuing iteration: 3.
Persuing iteration: 4.
Persuing iteration: 5.
Persuing iteration: 6.
Persuing iteration: 7.
Persuing iteration: 8.
Persuing iteration: 9.
